# 第 7 章 函数装饰器和闭包

---
本章的最终目标是解释清楚函数装饰器的工作原理,包括最简单的注册装饰器和较复杂的参数化装饰器。  
但是,在实现这一目标之前,我们要讨论下述话题:  
- Python 如何计算装饰器句法  
- Python 如何判断变量是不是局部的  
- 闭包存在的原因和工作原理  
- nonlocal 能解决什么问题

掌握这些基础知识后,我们可以进一步探讨装饰器:  

- 实现行为良好的装饰器  
- 标准库中有用的装饰器  
- 实现一个参数化装饰器  

---

### 7.1 装饰器基础知识
装饰器是可调用的对象,其参数是另一个函数(被装饰的函数)。  
装饰器可能会处理被装饰的函数,然后把它返回,或者将其替换成另一个函数或可调用对象。  

假如有个名为decorate的装饰器:

```python
@decorate
def target():
    print('running target()')
```

上述代码的效果与下述写法一样:

```python
def target():
    print('running target()')
    
target = decorate(target)
```

两种写法的最终结果一样:上述两个代码片段执行完毕后得到的 target 不一定是原来那个 target 函数,而是 decorate(target) 返
回的函数。

In [1]:
# 示例7-1 装饰器通常把函数替换成另一个函数

def deco(func):
    def inner():
        print('running inner()')
    return inner    # deco 返回 inner 函数对象

@deco
def target():    # 使用 deco 装饰器 target
    print('running target()')
    
target()    # 调用被装饰的 target 其实会运行 inner
print(target)    # 审查对象, 发现 target 现在是 inner 的引用

running inner()
<function deco.<locals>.inner at 0x7f9c9a7c7158>


严格来说,装饰器只是语法糖。如前所示,装饰器可以像常规的可调用对象那样调用,其参数是另一个函数。  
有时,这样做更方便,尤其是做元编程(在运行时改变程序的行为)时。  

综上,装饰器的一大特性是,能把被装饰的函数替换成其他函数。  
第二个特性是,装饰器在加载模块时立即执行。下一节会说明。  

### 7.2 Python何时执行装饰器
装饰器的一个关键特性是,它们在被装饰的函数定义之后立即运行。这通常是在导入时(即 Python 加载模块时),如示例 7-2 中的
registration.py 模块所示。

In [2]:
# 示例 7-2 registration.py 模块

registry = []    # registry 保存被 @register 装饰的函数引用

def register(func):    # register 的参数是一个函数
    print('running register(%s)' % func)    # 为了演示,显示被装饰的函数
    registry.append(func)    # 把 func 存入 registry
    return func    # 返回 func:必须返回函数;这里返回的函数与通过参数传入的一样

@register    # f1 和 f2 被 @register 装饰
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():    # f3 没有装饰
    print('running f3()')
        
def main() :    # main 显示 registry,然后调用 f1()、f2() 和 f3()
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    
if __name__ == '__main__':
    main()    # 只有把 registration.py 当作脚本运行时才调用 main()


    
# 注意,register 在模块中其他函数之前运行(两次)。
# 调用 register 时,传给它的参数是被装饰的函数,例如 <function f1 at 0x100631bf8>。
# 加载模块后, registry 中有两个被装饰函数的引用:f1 和 f2。这两个函数,以及 f3,只在 main 明确调用它们时才执行。

running register(<function f1 at 0x7f9ca806df28>)
running register(<function f2 at 0x7f9c9a7c76a8>)
running main()
registry -> [<function f1 at 0x7f9ca806df28>, <function f2 at 0x7f9c9a7c76a8>]
running f1()
running f2()
running f3()


---
如果导入 registration.py 模块(不作为脚本运行),输出如下:

In [3]:
import Tools.registration

running register(<function f1 at 0x7f9c9a7c7e18>)
running register(<function f2 at 0x7f9c9a7c77b8>)


In [4]:
Tools.registration.registry

[<function Tools.registration.f1()>, <function Tools.registration.f2()>]

这里主要想强调,函数装饰器在导入模块时立即执行,而被装饰的函数只在明确调用时运行。  
这突出了 Python 程序员所说的导入时和运行时之间的区别。  

考虑到装饰器在真实代码中的常用方式,示例 7-2 有两个不寻常的地方。
- 装饰器函数与被装饰的函数在同一个模块中定义。实际情况是,装饰器通常在一个模块中定义,然后应用到其他模块中的函数上。
- register 装饰器返回的函数与通过参数传入的相同。实际上,大多数装饰器会在内部定义一个函数,然后将其返回。

虽然示例 7-2 中的 register 装饰器原封不动地返回被装饰的函数,但是这种技术并非没有用处。  
很多 Python Web 框架使用这样的装饰器把函数添加到某种中央注册处,例如把 URL 模式映射到生成 HTTP 响应的函数上的注册处。  
这种注册装饰器可能会也可能不会修改被装饰的函数。  
下一节会举例说明。  

### 7.3 使用装饰器改进“策略”模式

使用注册装饰器可以改进 6.1 节中的电商促销折扣示例。

In [5]:
# 示例 7-3 promos 列表中的值使用 promotion 装饰器填充


promos = []    # promos 列表起初是空的

def promotion(promo_func):    # promotion 把 promo_func 添加到 promos 列表中,然后原封不动地将其返回
    promos.append(promo_func)
    return promo_func

@promotion    # 被 @promotion 装饰的函数都会添加到 promos 列表中
def fidelity(order):
    """  为积分1000或以上的顾客提供5%折扣 """
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """ 单个商品为20个或以上时提供10%折扣 """
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """ 订单中的不同商品达到10个或以上时提供7%折扣 """
    distinct_items = {item.product for item in orde.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):    # best_promos 无需修改,因为它依赖 promos 列表
    """ 选择可用的最佳折扣 """
    return max(promo(order) for promo in promos)

与 6.1 节给出的方案相比,这个方案有几个优点。
- 促销策略函数无需使用特殊的名称(即不用以 _promo 结尾)。
- @promotion 装饰器突出了被装饰的函数的作用,还便于临时禁用某个促销策略:只需把装饰器注释掉。
- 促销折扣策略可以在其他模块中定义,在系统中的任何地方都行,只要使用 @promotion 装饰即可。


### 7.4 变量作用域规则
在示例 7-4 中,我们定义并测试了一个函数,它读取两个变量的值:   
一个是局部变量 a,是函数的参数;另一个是变量 b,这个函数没有定义它。

In [6]:
# 示例 7-4 一个函数,读取一个局部变量和一个全局变量
import traceback

def f1(a):
    print('a=',a)
    print('b=',b)
    
try:
    f1(3)
except:
    traceback.print_exc()

a= 3


Traceback (most recent call last):
  File "<ipython-input-6-c523ebc043c3>", line 9, in <module>
    f1(3)
  File "<ipython-input-6-c523ebc043c3>", line 6, in f1
    print('b=',b)
NameError: name 'b' is not defined


In [7]:
# 出现错误并不奇怪。在示例 7-4 中,如果先给全局变量 b 赋值,然后再调用 f,那就不会出错:

def f1(a):
    print('a=',a)
    print('b=',b)
    
b = 6
f1(3)

a= 3
b= 6


下面看一个可能会让你吃惊的示例。   
看一下示例 7-5 中的 f2 函数。前两行代码与示例 7-4 中的 f1 一样,然后为 b 赋值,再打印它的值。可是,在赋值之前,第二个 print 失败了。  
示例 7-5 b 是局部变量,因为在函数的定义体中给它赋值了

In [8]:
b = 6

def f2(a):
    print('a=',a)
    print('b=',b)
    b = 9
try:    
    f2(3)
except:
    traceback.print_exc()

a= 3


Traceback (most recent call last):
  File "<ipython-input-8-0376c5fc39e2>", line 8, in <module>
    f2(3)
  File "<ipython-input-8-0376c5fc39e2>", line 5, in f2
    print('b=',b)
UnboundLocalError: local variable 'b' referenced before assignment


注意,首先输出了 3,这表明 print(a) 语句执行了。但是第二个语句 print(b) 执行不了。  
一开始我很吃惊,我觉得会打印 6,因为有个全局变量 b,而且是在 print(b) 之后为局部变量 b 赋值的。  

可事实是,Python 编译函数的定义体时,它判断 b 是局部变量,因为在函数中给它赋值了。生成的字节码证实了这种判断,Python 会尝试从本地环境获取 b。  
后面调用 f2(3) 时, f2 的定义体会获取并打印局部变量 a 的值,但是尝试获取局部变量 b 的值时,发现 b 没有绑定值。  

这不是缺陷,而是设计选择:Python 不要求声明变量,但是假定在函数定义体中赋值的变量是局部变量。  
这比 JavaScript 的行为好多了,JavaScript 也不要求声明变量,但是如果忘记把变量声明为局部变量(使用 var),可能会在不知情的情况下获取全局变量。  

In [9]:
# 如果在函数中赋值时想让解释器把 b 当成全局变量,要使用 global 声明:


b = 6

def f3(a):
    global b
    print('a=',a)
    print('b=',b)
    b = 9
    
f3(3)
print()
print('b=',b)
print()
f3(3)
print()
b = 30
print('b=',b)

a= 3
b= 6

b= 9

a= 3
b= 9

b= 30


### 7.5 闭包

闭包指延伸了作用域的函数,其中包含函数定义体中引用、但是不在定义体中定义的非全局变量。
函数是不是匿名的没有关系,关键是它能访问定义体之外定义的非全局变量

In [10]:
# 一个计算平均值的例子

# 用类的实例对象来实现
class Averager:
    
    def __init__(self):
        self.values = []
    
    
    def __call__(self, value):
        self.values.append(value)
        total = sum(self.values)
        return total / len(self.values)
    
avg = Averager()

print(avg(10))
print(avg(11))
print(avg(12))


# 用高阶函数实现
def make_averager():
    values = []
    
    def averager(value):
        values.append(value)
        total = sum(values)
        return total / len(values)
    
    return averager

avg = make_averager()

print()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0

10.0
10.5
11.0


### 7.6 nonlocal声明

前面实现 make_averager 函数的方法效率不高。在示例 7-9 中,我们把所有值存储在历史数列中,然后在每次调用 averager 时使用 sum 求和。更好的实现方式是,只存储目前的总值和元素个数,然后使用这两个数计算均值

In [11]:
# Python 3 引入了 nonlocal 声明。它的作用是把变量标记为自由变量,即使在函数中为变量赋予新值了,也会变成自由变量。
# 如果为 nonlocal 声明的变量赋予新值,闭包中保存的绑定会更新
# 注意: python 2 中没有 nonlocal 关键字


def make_averager():
    total, num = 0, 0
    
    def averager(value):
        nonlocal total, num
        total += value
        num += 1
        return total / num
    
    return averager

avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


### 7.7 实现一个简单的装饰器


In [12]:
# 示例 7-15 定义了一个装饰器,它会在每次调用被装饰的函数时计时,
# 然后把经过的时间、传入的参数和调用的结果打印出来。


import time

def clock(func):
    
    def clocked(*args):    # 定义内部函数 clocked,它接受任意个定位参数
        t0 = time.perf_counter()
        result = func(*args)    # 这行代码可用,是因为 clocked 的闭包中包含自由变量 func
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%08fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result   # 返回内部函数,取代被装饰的函数。演示了 clock 装饰器的用法
    
    return clocked

In [13]:
# 使用 clock 装饰器

@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! = ', factorial(6))

**************************************** Calling snooze(.123)
[0.123214s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.000001s] factorial(1) -> 1
[0.000063s] factorial(2) -> 2
[0.000109s] factorial(3) -> 6
[0.000163s] factorial(4) -> 24
[0.000216s] factorial(5) -> 120
[0.000265s] factorial(6) -> 720
6! =  720


**装饰器工作原理**

```python
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)
```
其实等价于:

```python
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

factorial = clock(factorial)
```

示例中实现的 clock 装饰器有几个缺点:不支持关键字参数,而且遮盖了被装饰函数的 \__name__ 和 \__doc__ 属性

In [14]:
# 改进后的装饰器


import functools


def clock2(func):
    
    @functools.wraps(func)    # functools.wraps 装饰器把相关的属性从 func 复制到 clocked 中
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_list = []
        if args:
            arg_list.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in kwargs.items()]
            arg_list.append(', '.join(pairs))
        arg_str = ', '.join(arg_list)
        print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result))
        return result
    
    return clocked

### 7.8 标准库中的装饰器

Python 内置了三个用于装饰方法的函数:property、classmethod 和 staticmethod

**使用functools.lru_cache做备忘**  

functools.lru_cache 是非常实用的装饰器,它实现了备忘(memoization)功能。  
这是一项优化技术,它把耗时的函数的结果保存起来,避免传入相同的参数时重复计算。  
LRU 三个字母是“LeastRecently Used”的缩写,表明缓存不会无限制增长,一段时间不用的缓存条目会被扔掉。  

In [15]:
# 生成第 n 个斐波纳契数,递归方式非常耗时


@clock2
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

print(fibonacci(6))

[0.00000173s] fibonacci(0) -> 0 
[0.00000123s] fibonacci(1) -> 1 
[0.00022533s] fibonacci(2) -> 1 
[0.00000092s] fibonacci(1) -> 1 
[0.00000115s] fibonacci(0) -> 0 
[0.00000120s] fibonacci(1) -> 1 
[0.00010870s] fibonacci(2) -> 1 
[0.00021043s] fibonacci(3) -> 2 
[0.00052088s] fibonacci(4) -> 3 
[0.00000053s] fibonacci(1) -> 1 
[0.00000060s] fibonacci(0) -> 0 
[0.00000092s] fibonacci(1) -> 1 
[0.00008530s] fibonacci(2) -> 1 
[0.00015832s] fibonacci(3) -> 2 
[0.00000049s] fibonacci(0) -> 0 
[0.00000127s] fibonacci(1) -> 1 
[0.00008930s] fibonacci(2) -> 1 
[0.00000119s] fibonacci(1) -> 1 
[0.00000120s] fibonacci(0) -> 0 
[0.00000128s] fibonacci(1) -> 1 
[0.00009212s] fibonacci(2) -> 1 
[0.00018130s] fibonacci(3) -> 2 
[0.00036057s] fibonacci(4) -> 3 
[0.00060313s] fibonacci(5) -> 5 
[0.00120227s] fibonacci(6) -> 8 
8


In [16]:
# 使用缓存实现,速度更快


@functools.lru_cache()   # 这里叠放了装饰器:@lru_cache() 应用到 @clock 返回的函数上
@clock2
def fibonacci_fast(n):
    if n < 2:
        return n
    return fibonacci_fast(n - 2) + fibonacci_fast(n - 1)

print(fibonacci_fast(6))

[0.00000096s] fibonacci_fast(0) -> 0 
[0.00000129s] fibonacci_fast(1) -> 1 
[0.00020500s] fibonacci_fast(2) -> 1 
[0.00000257s] fibonacci_fast(3) -> 2 
[0.00032170s] fibonacci_fast(4) -> 3 
[0.00000272s] fibonacci_fast(5) -> 5 
[0.00045557s] fibonacci_fast(6) -> 8 
8


特别要注意,lru_cache 可以使用两个可选的参数来配置。它的签名是:

```python
@functools.lru_cache(maxsize=128, typed=False)
```

maxsize 参数指定存储多少个调用的结果。缓存满了之后,旧的结果会被扔掉,腾出空间。为了得到最佳性能,maxsize 应该设为 2 的幂。  
typed 参数如果设为 True,把不同参数类型得到的结果分开保存,即把通常认为相等的浮点数和整数参数(如 1 和 1.0)区分开。  
顺便说一下,因为 lru_cache 使用字典存储结果,而且键根据调用时传入的定位参数和关键字参数创建,所以被 lru_cache 装饰的函数,它的所有参数都必须是可散列的。

**单分派泛函数**

In [17]:
import html


# 假设我们在开发一个调试 Web 应用的工具,我们想生成 HTML,显示不同类型的 Python 对象。
# 我们可能会编写这样的函数:
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</per>'.format(content)

这个函数适用于任何 Python 类型,但是现在我们想做个扩展,让它使用特别的方式显示某些类型。
- `str:把内部的换行符替换为 '<br>\n';不使用 <pre>,而是使用 <p>`
- `int:以十进制和十六进制显示数字。`  
- `list:输出一个 HTML 列表,根据各个元素的类型进行格式化。`  

因为 Python 不支持重载方法或函数,所以我们不能使用不同的签名定义htmlize 的变体,也无法使用不同的方式处理不同的数据类型。  
在Python 中,一种常见的做法是把htmlize变成一个分派函数,使用一串 if/elif/elif,调用专门的函数,如htmlize_str、htmlize_int,等等。<br>这样不便于模块的用户扩展,还显得笨拙:时间一长,分派函数 htmlize 会变得很大,而且它与各个专门函数之间的耦合也很紧密。 

Python 3.4 新增的 **functools.singledispatch** 装饰器可以把整体方案拆分成多个模块,甚至可以为你无法修改的类提供专门的函数。  
使用@singledispatch 装饰的普通函数会变成泛函数(generic function):  
根据第一个参数的类型,以不同方式执行相同操作的一组函数。具体做法参见示例

In [18]:
# singledispatch 创建一个自定义的 htmlize.register 装饰器,把多个函数绑在一起组成一个泛函数


from functools import singledispatch
from collections import abc
import numbers
import html


@singledispatch    # @singledispatch 标记处理 object 类型的基函数
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)


@htmlize.register(str)    # 各个专门函数使用 @«base_function».register(«type») 装饰
def _(text):              # 专门函数的名称无关紧要;_ 是个不错的选择,简单明了
    content = html.escape(text).replace('\n', '\<br>\n')
    return '<p>{}</p>'.format(content)


@htmlize.register(numbers.Integral)    # 为每个需要特殊处理的类型注册一个函数。numbers.Integral 是 int 的虚拟超类
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)


@htmlize.register(tuple)    # 可以叠放多个 register 装饰器,让同一个函数支持不同类型
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

只要可能,**注册的专门函数应该处理抽象基类**(如 numbers.Integral 和 abc.MutableSequence),不要处理具体实现(如 int 和 list)。  
这样,代码支持的兼容类型更广泛。

In [19]:
# 测试单分派泛函数


print(htmlize({1, 2, 3}))
print(htmlize(abs))
print(htmlize('Heimlich & Co.\n- a game'))
print(htmlize(42))
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>{1, 2, 3}</pre>
<pre>&lt;built-in function abs&gt;</pre>
<p>Heimlich &amp; Co.\<br>
- a game</p>
<pre>42 (0x2a)</pre>
<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


### 7.9 叠放装饰器

把 @d1 和 @d2 两个装饰器按顺序应用到 f 函数上,作用相当于 f = d1(d2(f))

```python
@d1
@d2
def f():
    print('f')
```

等同于

```python
def f():
    print('f')
    
f = d1(d2(f))
```

### 7.10 参数化装饰器

Python 把被装饰的函数作为第一个参数传给装饰器函数。  
那怎么让装饰器接受其他参数呢?答案是:创建一个装饰器工厂函数,把参数传给它,返回一个装饰器,然后再把它应用到要装饰的函数上

In [20]:
# 示例如下


registry = []


def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x7f9ca806de18>)
running main()
registry -> [<function f1 at 0x7f9ca806de18>]
running f1()


**一个参数化的注册装饰器**  
为了便于启用或禁用 register 执行的函数注册功能,我们为它提供一个可选的 active 参数,设为 False 时,不注册被装饰的函数。  
实现方式参见示例。从概念上看,这个新的 register 函数不是装饰器,而是装饰器工厂函数。  
调用它会返回真正的装饰器,这才是应用到目标函数上的装饰器。  

In [21]:
# 为了接受参数,新的 register 装饰器必须作为函数调用

registry = set()    # registry 现在是一个 set 对象,这样添加和删除函数的速度更快
def register(active=True):    # register 接受一个可选的关键字参数
    def decorate(func):       # decorate 这个内部函数是真正的装饰器;注意,它的参数是一个函数
        print('running register(active=%s)->decorate(%s)' % (active, func))
        if active:            # 只有 active 参数的值(从闭包中获取)是 True 时才注册 func
            registry.add(func)
        else:                 # 如果 active 不为真,而且 func 在 registry 中,那么把它删除
            registry.discard(func)
            
        return func           # decorate 是装饰器,必须返回一个函数
    return decorate           # register 是装饰器工厂函数,因此返回 decorate


@register(active=False)       # @register 工厂函数必须作为函数调用,并且传入所需的参数
def f1():
    print('running f1()')
    
    
@register()                   # 即使不传入参数,register 也必须作为函数调用 (@register()),即要返回真正的装饰器 decorate
def f2():
    print('running f2()')
    

def f3():
    print('running f3()')
    
    
f1()
f2()
f3()
print(registry)
print(register()(f3))    # register() 表达式返回 decorate,然后把它应用到 f3 上
print(registry)          # 前一行把 f3 添加到 registry 中
print(register(active=False)(f2))    # 这次调用从 registry 中删除 f2
print(registry)          # 确认 registry 中只有 f3

running register(active=False)->decorate(<function f1 at 0x7f9c9a7c7950>)
running register(active=True)->decorate(<function f2 at 0x7f9ca806df28>)
running f1()
running f2()
running f3()
{<function f2 at 0x7f9ca806df28>}
running register(active=True)->decorate(<function f3 at 0x7f9c9a7c7730>)
<function f3 at 0x7f9c9a7c7730>
{<function f2 at 0x7f9ca806df28>, <function f3 at 0x7f9c9a7c7730>}
running register(active=False)->decorate(<function f2 at 0x7f9ca806df28>)
<function f2 at 0x7f9ca806df28>
{<function f3 at 0x7f9c9a7c7730>}


**参数化clock装饰器**

本节再次探讨 clock 装饰器,为它添加一个功能:让用户传入一个格式字符串,控制被装饰函数的输出。

In [22]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock3(fmt=DEFAULT_FMT):   # clock3 是参数化装饰器工厂函数
    def decorate(func):        # decorate 是真正的装饰器
        def clocked(*_args):   # clocked 包装被装饰的函数
            t0 = time.time()
            _result = func(*_args)    # _result 是被装饰的函数返回的真正结果
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)    # _args 是 clocked 的参数,args 是用于显示的字符串
            result = repr(_result)    # result 是 _result 的字符串表示形式,用于显示
            print(fmt.format(**locals()))    # 这里使用 **locals() 是为了在 fmt 中引用 clocked 的局部变量
            return _result    # clocked 会取代被装饰的函数,因此它应该返回被装饰的函数返回的值
        return clocked    # decorate 返回 clocked
    return decorate    # clock 返回 decorate


# 在这个模块中测试,不传入参数调用 clock(),因此应用的装饰器使用默认的格式 str
@clock3()
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

[0.12323141s] snooze(0.123) -> None
[0.12317133s] snooze(0.123) -> None
[0.12323403s] snooze(0.123) -> None


In [23]:
@clock3('{name}: {elapsed:.8f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123) 

snooze: 0.12318540s
snooze: 0.12319803s
snooze: 0.12320018s


In [24]:
@clock3('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
